In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from QBUS2820 import rmse_jack, r2_jack 
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from ExtraCode import getResultTable
from sklearn.model_selection import cross_val_score
from QBUS2820 import forward
from sklearn.linear_model import ElasticNet
from QBUS2820 import pcrCV
from QBUS2820 import plsCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
#Stop warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [3]:
data = pd.read_csv('Train6_2.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')
y_train = np.log(y_train)



In [4]:
pred = []
method = []

In [75]:
pcr=pcrCV(final_train, y_train)
predpcr = pcr.predict(final_test)
method.append('PCR')
pred.append(np.exp(predpcr))


In [ ]:
neighbor = np.arange(1,101)
optimalk = 0
low = np.inf
for i in neighbor:
    knn1 = KNeighborsRegressor(n_neighbors=i) 
    predictions = knn1.fit(final_train, y_train)
    predictionsfin = predictions.predict(final_test)
    rmse = np.sqrt(mean_squared_error(y_test, np.exp(predictionsfin)))
    if rmse < low:
        low = rmse
        optimalk = i
print('Optimal k is {}'.format(optimalk))

In [ ]:
knn1 = KNeighborsRegressor(n_neighbors=optimalk) 
predictions = knn1.fit(final_train, y_train)
predknn = knn1.predict(final_test)
method.append('KNN with optimal {}'.format(optimalk))
pred.append(np.exp(predknn))

### Testing Models

In [5]:
#Forward selection
fwd = forward()
fwd.fit(final_train, y_train)
predforward = fwd.predict(final_test)
method.append('Forward')
pred.append(np.exp(predforward))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost')
pred.append(np.exp(predFinalBoost))

regr = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)
regr = regr.fit(final_train,y_train)
adapred = regr.predict(final_test)
method.append('AdaBoost')
pred.append(np.exp(adapred))

In [6]:
xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
testFinaldmat = xgb.DMatrix(final_test)
xgpred = final_gb.predict(testFinaldmat)
method.append('XGBoost')
pred.append(np.exp(xgpred))

In [7]:
pred2 = (np.exp(xgpred)+np.exp(adapred)+np.exp(predFinalBoost))/3
pred.append(pred2)
method.append('XGBoost, ada, GB')

In [79]:
pred3 = (np.exp(xgpred)+np.exp(predFinalBoost))/2

pred5 = (np.exp(xgpred)+np.exp(adapred))/2
pred.append(pred3)
pred.append(pred5)
method.append('xg GB')
method.append('xg adaboost')

In [8]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(final_train,y_train)

predFinalExtRandomForestlad = regr2.predict(final_test)
pred.append(np.exp(predFinalExtRandomForestlad))
method.append('Random Forest')

In [81]:
pred5 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad)+np.exp(predFinalBoost))/3
pred.append(pred5)
method.append('XGBoost, random tree, GB')

In [82]:
pred6 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad))/2
pred.append(pred6)
method.append('xg Random tree')

In [9]:
pred10 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad))/3
pred.append(pred10)
method.append('XGBoost, GBoost Random Forest')

In [10]:
#LASSO
lasso = Lasso(alpha = 1)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(final_test)
method.append('LASSO')
pred.append(np.exp(pred_L))

In [11]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(final_train, y_train)
y_pred_rg = (rg.predict(final_test))
y_pred_rg

pred.append(np.exp(y_pred_rg))
method.append('Ridge alpha 10')

In [86]:
pred11 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4
pred.append(pred11)
method.append('XGBoost, GBoost Random Forest Ridge')

In [87]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred_L))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest LASSO')

In [12]:
regr = ElasticNet(random_state=0, alpha = 1)
    
regr.fit(final_train, y_train)    
pred13 = regr.predict(final_test)
pred.append(np.exp(pred13))
method.append('ENET')

In [13]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred13))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest ENET')

In [90]:
pred13 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg))/3
pred.append(pred13)
method.append('XGBoost, GBoost Ridge')

In [91]:
pred14 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward))/4
pred.append(pred14)
method.append('XGBoost, GBoost Ridge Forward')

In [92]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(predFinalExtRandomForestlad))/5
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward Tree')

In [93]:
pred16 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(pred_L))/5
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward Lasso')

In [94]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+ np.exp(predFinalExtRandomForestlad))/3
pred.append(pred15)
method.append('XGBoost, GBoost, RandomTree')

In [14]:

pred15 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward 30 30 20 20')

In [15]:
pred16 = ((0.4*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward 40 40 10 10')

In [97]:
pred17 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred17)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [16]:
pred18 = ((0.35*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg))+(0.15*np.exp(predforward)))
pred.append(pred18)
method.append('XGBoost, GBoost Ridge Forward 35 35 15 15')

In [99]:
pred19 = ((0.25*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred19)
method.append('XGBoost, GBoost Ridge Forward 25 35 20 20')

In [100]:
pred20 = ((0.2*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred20)
method.append('XGBoost, GBoost Ridge Forward 20 40 20 20')

In [101]:
pred21 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred21)
method.append('XGBoost, GBoost Ridge Forward 20 30 20 30')

In [102]:
pred22 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred22)
method.append('XGBoost, GBoost Ridge Forward 20 30 30 20')

In [103]:
pred23 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred23)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [104]:
pred24 = ((0.3*np.exp(xgpred))+(0.5*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 30 50 10 10')

In [105]:
pred24 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.4*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 20 20 20 40')

In [106]:
pred25 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred25)
method.append('XGBoost, GBoost Ridge Forward 30 30 30 10')

In [107]:
pred26 = ((0.26*np.exp(xgpred))+(0.27*np.exp(predFinalBoost))+(0.27*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred26)
method.append('XGBoost, GBoost Ridge Forward 26 27 27 20')

In [108]:
pred27 = ((0.23*np.exp(xgpred))+(0.24*np.exp(predFinalBoost))+(0.24*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred27)
method.append('XGBoost, GBoost Ridge Forward 23 24 24 30')

In [26]:
pred21 = ((0.15*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.35*np.exp(predforward))+ (0.05*np.exp(predFinalExtRandomForestlad)))
pred.append(pred21)
method.append('XGBoost, GBoost Ridge Forward RandomForest 15 35 10 35 5')

In [27]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
Forward,25198.791,6787.831,0.870,0.064,13627.500,0.870
GBoost,20440.707,3388.004,0.915,0.019,13586.876,0.915
AdaBoost,26380.119,3688.592,0.858,0.025,18322.320,0.858
XGBoost,20400.772,3115.510,0.915,0.018,13657.482,0.915
"XGBoost, ada, GB",20756.360,3675.439,0.912,0.021,13747.382,0.912
Random Forest,25932.501,3558.011,0.863,0.023,18005.818,0.863
"XGBoost, GBoost Random Forest",20550.617,3636.619,0.914,0.021,13589.502,0.914
LASSO,35167.048,7697.295,0.747,0.103,20194.557,0.747
Ridge alpha 10,22222.019,5879.099,0.899,0.049,12783.095,0.899
ENET,34608.305,8504.880,0.755,0.109,18945.275,0.755


In [24]:
def getResultTable(rows, predictions):
    columns=['Test RMSE', 'SE', 'Jack R2', 'SE', 'MAE', 'R-square']
    results=pd.DataFrame(0.0, columns=columns, index=rows)

    for row,pred in zip(range(0,len(rows)),predictions):
        results.iloc[row,0], results.iloc[row,1] = rmse_jack(y_test, pred)
        results.iloc[row,2], results.iloc[row,3] = (r2_jack(y_test, pred))
        results.iloc[row,4] = mean_absolute_error(y_test, pred)
        results.iloc[row,5] = r2_score(y_test,pred)
    return results.round(3)

## Kaggle Predictions

In [119]:
data = pd.read_csv('Train_del_332.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test_del_332.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [120]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [121]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [122]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [123]:
#Forward selection
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)

In [69]:
finalprediction = ((0.15*np.exp(prediction))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg) + (0.35*np.exp(predforward))))

In [49]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction.to_csv("Day18_1.csv", index=False)

In [50]:
prediction

,Id,Prediction
0,1,90807.229520
1,2,163753.280426
2,3,134565.541332
3,4,186791.822738
4,5,151033.594125
5,6,320050.960626
6,7,131325.897713
7,8,281905.802572
8,9,118401.016685
9,10,133941.742398


In [111]:
data = pd.read_csv('Train_del&bin20.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test_del&bin20.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [70]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction.to_csv("Day18_2.csv", index=False)

In [71]:
prediction

,Id,Prediction
0,1,95325.273830
1,2,167363.188087
2,3,137044.936774
3,4,182289.933429
4,5,153357.450700
5,6,332427.369852
6,7,131618.201473
7,8,258126.239846
8,9,119280.819446
9,10,132829.365880


In [116]:
finalprediction3 = ((0.25*np.exp(prediction))+(0.25*np.exp(predFinalBoost))+(0.25*np.exp(y_pred_rg) + (0.25*np.exp(predforward))))

In [117]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction3 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction3})
#Saving results into CSV file 
prediction3.to_csv("Day18_3.csv", index=False)

In [118]:
prediction3

,Id,Prediction
0,1,95508.953541
1,2,167805.209174
2,3,137128.663733
3,4,182275.655525
4,5,154853.808765
5,6,331031.073723
6,7,132021.923390
7,8,262166.501665
8,9,119710.121670
9,10,132216.502149


In [124]:
#LASSO
lasso = Lasso(alpha = 1)
lasso.fit(data, np.ravel(y_price)) 
pred_L_final = lasso.predict(kaggle)

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [140]:
np.exp(pred_L_final)

array([ 117625.3068124 ,  154406.58809745,  135270.46006291, ...,
        106935.400622  ,  164524.5301393 ,  224045.45467818])

In [125]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(data,y_price)

predFinalExtRandomForestlad = regr2.predict(kaggle)

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [141]:
finalprediction4 = ((np.exp(predFinalExtRandomForestlad)*(1/5))+(np.exp(prediction)*(1/5))+(np.exp(predFinalBoost)*(1/5))+(np.exp(y_pred_rg)*(1/5)) + (np.exp(predforward)*(1/5)))

In [142]:
finalprediction4

array([  97623.67532531,  159917.41864414,  133822.27050019, ...,
         92461.51084685,  189523.84021274,  215180.31456205])

In [143]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction4 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction4})
#Saving results into CSV file 
prediction4.to_csv("Day18_4.csv", index=False)